In [9]:
# %load "test (2).py"
import numpy as np
import csv
import glob
import matplotlib as plt
import matplotlib.pyplot as plt
# %matplotlib inline
import xarray as xr
import seaborn as sns;
sns.set(color_codes=True)
plt.style.use('ggplot')
import netCDF4 as nc
# from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors

from pylab import *
import os
from matplotlib import ticker
from netCDF4 import Dataset
import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import numpy as np
import datetime
import calendar as cal
from matplotlib.colors import ListedColormap, BoundaryNorm
import cmocean
#
# data = xr.open_dataset('/Volumes/WorkDrive/melt_dates/seaiceconc.nc') # opening sit for xarray
# ## Arange months up to Aug to December ##
# def is_months(month):
#     return (month >= 8) & (month <= 12)
# data = data.sel(time=is_months(data['time.month']))

# ## Select SIC variable ##
# sic_xr = data.SI_12km_SH_ICECON_DAY_SpPolarGrid12km
# latitude = data.GridLat_SpPolarGrid12km
# longitude = data.GridLon_SpPolarGrid12km
#
# ## Here we want all the values that are over 100 (missing or land mask) to be set to nan ##
# sic_xr = sic_xr.where(sic_xr < 101,np.nan)

## Because the desired output is one slice/ image/grid for each year, I will process year by year
def continuous_meet(cond, window_size, dim):
    """
    Continuously meet a given condition along a dimension.
    """
    _found = cond.rolling(dim={'time': window_size},
                          center=True).sum(skipna=True).fillna(False).astype(np.float)

    print('--->>>>',_found)
    detected = np.array(
        _found.rolling(dim={'time': window_size})
        .reduce(lambda a, axis: (a == window_size).any(axis=axis))
        .fillna(False)
        .astype(bool)
    )

    indices = (detected * np.arange(detected.shape[0]).reshape(detected.shape[0], 1, 1))
    indices[indices == 0] = detected.shape[0]
    output = indices.argmin(axis=0)

    return xr.DataArray(output)

sub_data = xr.open_dataset('daily_sic_subset.nc')
print(sub_data)
ice =  sub_data.__xarray_dataarray_variable__
ice.plot()


<xarray.Dataset>
Dimensions:                                (time: 150,
                                            YDim_SpPolarGrid12km: 664,
                                            XDim_SpPolarGrid12km: 632)
Coordinates:
  * time                                   (time) datetime64[ns] 2012-08-01 ....
    GridLon_SpPolarGrid12km                (YDim_SpPolarGrid12km, XDim_SpPolarGrid12km) float64 ...
    GridLat_SpPolarGrid12km                (YDim_SpPolarGrid12km, XDim_SpPolarGrid12km) float64 ...
Dimensions without coordinates: YDim_SpPolarGrid12km, XDim_SpPolarGrid12km
Data variables:
    SI_12km_SH_ICECON_DAY_SpPolarGrid12km  (time, YDim_SpPolarGrid12km, XDim_SpPolarGrid12km) int32 ...
Attributes:
    CDI:             Climate Data Interface version 1.9.9 (https://mpimet.mpg...
    Conventions:     None
    creation_date:   Mon 19 Jul 2021 08:05:55 PDT
    NCL_Version:     6.6.2
    system:          Darwin Fridas-MBP.attlocal.net 20.5.0 Darwin Kernel Vers...
    hdfeos5_source: 

AttributeError: 'Dataset' object has no attribute '__xarray_dataarray_variable__'

In [6]:
time

<module 'time' (built-in)>

In [8]:
continuous_meet(30, 5, dim='time')

AttributeError: 'int' object has no attribute 'rolling'